In [1]:
!gdown 1d7JABk4jViI-USjLsWmhGkvzi8uQIL5C

Downloading...
From: https://drive.google.com/uc?id=1d7JABk4jViI-USjLsWmhGkvzi8uQIL5C
To: /content/data.zip
100% 151k/151k [00:00<00:00, 107MB/s]


In [2]:
!unzip ./data.zip

Archive:  ./data.zip
   creating: data/
  inflating: __MACOSX/._data         
  inflating: data/restaurants_train.csv  
  inflating: __MACOSX/data/._restaurants_train.csv  
  inflating: data/restaurants_test.csv  
  inflating: __MACOSX/data/._restaurants_test.csv  


In [4]:
import pandas as pd
df = pd.read_csv('/content/data/restaurants_test.csv')
df.head()

,Tokens,Tags,Polarities
0,"['The', 'bread', 'is', 'top', 'notch', 'as', '...","[0, 1, 0, 0, 0, 0, 0, 0]","[-1, 2, -1, -1, -1, -1, -1, -1]"
1,"['I', 'have', 'to', 'say', 'they', 'have', 'on...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 2, 2,..."
2,"['Good', 'is', 'always', 'fresh', 'and', 'hot'...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[2, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]"
3,"['Did', 'I', 'mention', 'that', 'the', 'coffee...","[0, 0, 0, 0, 0, 1, 0, 0, 0]","[-1, -1, -1, -1, -1, 2, -1, -1, -1]"
4,"['Certainly', 'not', 'the', 'best', 'such', 'i...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."


- Ở đây, ví dụ trong câu The bread is top notch,...
các nhãn tags được gắn là 0, 1, 0, 0 trong đó 0 tức là non-aspect class, trong khi 1 là aspect class.

- Các nhãn được sinh ra trong polarities có thể hiểu là -1: không mang cảm xúc gì, 1 mang cảm xúc tiêu cực, 2 mang cảm xúc tích cực

In [14]:
df.iloc[0].values

array(["['The', 'bread', 'is', 'top', 'notch', 'as', 'well']",
       '[0, 1, 0, 0, 0, 0, 0, 0]', '[-1, 2, -1, -1, -1, -1, -1, -1]'],
      dtype=object)

##**Dataset**

In [15]:
import torch
from torch.utils.data import Dataset

class ABSADataset(Dataset):
    def __init__(self, df, tokenizer):
        self.df = df
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        tokens, tags, pols = self.df.iloc[idx, :3].values

        tokens = tokens.replace("'", "").strip("][").split(', ')
        tags = tags.strip('][').split(', ')
        pols = pols.strip('][').split(', ')

        bert_tokens = []
        bert_tags = []
        bert_pols = []
        for i in range(len(tokens)):
            t = self.tokenizer.tokenize(tokens[i])
            bert_tokens += t
            bert_tags += [int(tags[i])]*len(t)
            bert_pols += [int(pols[i])]*len(t)

        bert_ids = self.tokenizer.convert_tokens_to_ids(bert_tokens)

        ids_tensor = torch.tensor(bert_ids)
        tags_tensor = torch.tensor(bert_tags)
        pols_tensor = torch.tensor(bert_pols)
        return bert_tokens, ids_tensor, tags_tensor, pols_tensor

    def __len__(self):
        return len(self.df)

In [17]:
from transformers import BertTokenizer

model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [18]:
tokenizer.all_special_tokens

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']

In [19]:
tokenizer.cls_token_id

101

In [20]:
tokenizer.sep_token_id

102

In [21]:
train_df = pd.read_csv('/content/data/restaurants_train.csv')
test_df = pd.read_csv('/content/data/restaurants_test.csv')
train_ds = ABSADataset(train_df, tokenizer)
test_ds = ABSADataset(test_df, tokenizer)

In [22]:
len(train_ds)

3602

In [23]:
next(iter(train_ds))

(['but', 'the', 'staff', 'was', 'so', 'horrible', 'to', 'us'],
 tensor([2021, 1996, 3095, 2001, 2061, 9202, 2000, 2149]),
 tensor([0, 0, 1, 0, 0, 0, 0, 0]),
 tensor([-1, -1,  0, -1, -1, -1, -1, -1]))

In [35]:
ids_tensors = [s[1] for s in test_ds]


AttributeError: 'list' object has no attribute 'shape'

In [38]:
len(ids_tensors)
print(ids_tensors[0])
print(ids_tensors[1])


tensor([ 1996,  7852,  2003,  2327, 18624,  2004,  2092])
tensor([ 1045,  2031,  2000,  2360,  2027,  2031,  2028,  1997,  1996,  6638,
        22199,  6959,  2335,  1999,  1996,  2103])


In [43]:
ids_tensors = pad_sequence(ids_tensors, batch_first=True)

masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long)
masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1)
masks_tensors

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])

In [24]:
from torch.nn.utils.rnn import pad_sequence

def padding(samples):
    ids_tensors = [s[1] for s in samples]
    ids_tensors = pad_sequence(ids_tensors, batch_first=True)

    tags_tensors = [s[2] for s in samples]
    tags_tensors = pad_sequence(tags_tensors, batch_first=True)

    pols_tensors = [s[3] for s in samples]
    pols_tensors = pad_sequence(pols_tensors, batch_first=True)

    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1)

    return ids_tensors, tags_tensors, pols_tensors, masks_tensors

In [39]:
from torch.nn.utils.rnn import pad_sequence
a = torch.ones(25)
b = torch.ones(23)
c = torch.ones(15)
pad_sequence([a, b, c], batch_first=True).size()

torch.Size([3, 25])

In [40]:
from torch.utils.data import DataLoader

batch_size = 32
train_loader = DataLoader(
    train_ds, batch_size=batch_size, shuffle=True, collate_fn=padding
)
test_loader = DataLoader(
    test_ds, batch_size=batch_size, shuffle=True, collate_fn=padding
)

In [41]:
next(iter(train_loader))

(tensor([[ 1998,  2348,  7251,  ...,     0,     0,     0],
         [ 1999,  1996,  2060,  ...,  3737,  1998, 11712],
         [ 2049,  3295,  2003,  ...,     0,     0,     0],
         ...,
         [ 2002,  2245,  2008,  ...,     0,     0,     0],
         [ 2010,  2173,  2003,  ...,     0,     0,     0],
         [ 3580,  7224,  1996,  ...,     0,     0,     0]]),
 tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 1, 0, 1],
         [0, 1, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 1, 0,  ..., 0, 0, 0]]),
 tensor([[-1, -1, -1,  ...,  0,  0,  0],
         [-1, -1, -1,  ...,  0, -1, -1],
         [-1,  2, -1,  ...,  0,  0,  0],
         ...,
         [-1, -1, -1,  ...,  0,  0,  0],
         [-1, -1, -1,  ...,  0,  0,  0],
         [-1, -1, -1,  ...,  0,  0,  0]]),
 tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ...

##**Model**

In [44]:
from transformers import BertModel

class ABTEBert(torch.nn.Module):
    def __init__(self, model_name):
        super(ABTEBert, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.linear = torch.nn.Linear(self.bert.config.hidden_size, 3)
        self.loss_fn = torch.nn.CrossEntropyLoss()

    def forward(self, ids_tensors, masks_tensors, tags_tensors):
        bert_outputs= self.bert(
            input_ids=ids_tensors, attention_mask=masks_tensors, return_dict=False
            )
        bert_outputs = bert_outputs[0]

        linear_outputs = self.linear(bert_outputs)
        if tags_tensors is not None:
            tags_tensors = tags_tensors.view(-1)
            linear_outputs_ = linear_outputs.view(-1,3)
            loss = self.loss_fn(linear_outputs_, tags_tensors)
            return loss, linear_outputs
        else:
            return linear_outputs

In [45]:
model = ABTEBert(model_name)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [46]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [47]:
model.to(device)

ABTEBert(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=T

In [48]:
import time
import numpy as np
from sklearn.metrics import classification_report

def train_epoch(model, optimizer, train_loader, device):
    losses = []
    for batch in (train_loader):
        ids_tensors, tags_tensors, _, masks_tensors = batch
        ids_tensors = ids_tensors.to(device)
        tags_tensors = tags_tensors.to(device)
        masks_tensors = masks_tensors.to(device)

        loss, _ = model(
            ids_tensors=ids_tensors,
            masks_tensors=masks_tensors,
            tags_tensors=tags_tensors
        )
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    return sum(losses)/len(losses)

def evaluate_epoch(model, valid_loader, device):
    losses = []

    preds, labels = [], []
    with torch.no_grad():
        for batch in (valid_loader):
            ids_tensors, tags_tensors, _, masks_tensors = batch
            ids_tensors = ids_tensors.to(device)
            tags_tensors = tags_tensors.to(device)
            masks_tensors = masks_tensors.to(device)

            loss, outputs = model(
                ids_tensors=ids_tensors,
                masks_tensors=masks_tensors,
                tags_tensors=tags_tensors
            )
            losses.append(loss.item())

            _, p = torch.max(outputs, dim=2)
            preds += list([int(j) for i in p for j in i ])
            labels += list([int(j) for i in tags_tensors for j in i ])

    acc = np.mean(np.array(preds) == np.array(labels))
    return sum(losses)/len(losses), acc

def train(model, model_name, save_model, optimizer, train_loader, valid_loader, num_epochs, device):
    train_losses = []
    eval_accs, eval_losses = [], []
    best_loss_eval = 100
    times = []
    for epoch in range(1, num_epochs+1):
        epoch_start_time = time.time()
        # Training
        train_loss = train_epoch(model, optimizer, train_loader, device)
        train_losses.append(train_loss)

        # Evaluation
        eval_loss, eval_acc = evaluate_epoch(model, valid_loader, device)
        eval_accs.append(eval_acc)
        eval_losses.append(eval_loss)

        # Save best model
        if eval_loss < best_loss_eval:
            torch.save(model.state_dict(), save_model + f'/{model_name}.pt')

        times.append(time.time() - epoch_start_time)
        # Print loss, acc end epoch
        print("-" * 59)
        print(
            "| End of epoch {:3d} | Time: {:5.2f}s | Train Loss {:8.3f} "
            "| Valid Accuracy {:8.3f} | Valid Loss {:8.3f} ".format(
                epoch, time.time() - epoch_start_time, train_loss, eval_acc, eval_loss
            )
        )
        print("-" * 59)

    # Load best model
    model.load_state_dict(torch.load(save_model + f'/{model_name}.pt'))
    model.eval()
    metrics = {
        'train_loss': train_losses,
        'valid_accuracy': eval_accs,
        'valid_loss': eval_losses,
        'time': times
    }
    return model, metrics

In [49]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

In [50]:
loss = train_epoch(model, optimizer, train_loader, device)
loss

0.26829841864847503

In [51]:
loss, acc = evaluate_epoch(model, test_loader, device)
loss, acc

(0.22941152538572038, 0.9146782920088747)

##**Training**

In [52]:
!mkdir "./model"

In [53]:
save_model = "./model"
model = ABTEBert(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
num_epochs = 5
best_model, metrics = train(
    model, model_name, save_model, optimizer, train_loader, test_loader, num_epochs, device
)

-----------------------------------------------------------
| End of epoch   1 | Time: 35.58s | Train Loss    0.260 | Valid Accuracy    0.915 | Valid Loss    0.219 
-----------------------------------------------------------
-----------------------------------------------------------
| End of epoch   2 | Time: 38.29s | Train Loss    0.148 | Valid Accuracy    0.918 | Valid Loss    0.222 
-----------------------------------------------------------
-----------------------------------------------------------
| End of epoch   3 | Time: 44.26s | Train Loss    0.094 | Valid Accuracy    0.915 | Valid Loss    0.251 
-----------------------------------------------------------
-----------------------------------------------------------
| End of epoch   4 | Time: 38.45s | Train Loss    0.048 | Valid Accuracy    0.914 | Valid Loss    0.303 
-----------------------------------------------------------
-----------------------------------------------------------
| End of epoch   5 | Time: 42.60s | Trai

##**Prediction**

In [54]:
def predict(best_model, sentence, device):
    word_pieces = list(tokenizer.tokenize(sentence))
    input_ids = tokenizer.convert_tokens_to_ids(word_pieces)
    input_tensor = torch.tensor([input_ids]).to(device)

    with torch.no_grad():
        outputs = model(input_tensor, None, None)
        _, predictions = torch.max(outputs, dim=2)

    predictions = predictions[0].tolist()
    return word_pieces, predictions, outputs

In [55]:
sentence = " ".join(test_df.iloc[0]["Tokens"].replace("'", "").strip("][").split(', '))
predict(best_model, sentence, device)

(['the', 'bread', 'is', 'top', 'notch', 'as', 'well'],
 [0, 1, 0, 0, 0, 0, 0],
 tensor([[[ 5.1472, -1.5992, -2.2205],
          [-1.8609,  4.6382, -2.9456],
          [ 5.2254, -2.3421, -2.3487],
          [ 5.4267, -2.1419, -2.6003],
          [ 5.0855, -2.2131, -2.4447],
          [ 5.4412, -2.3073, -2.6976],
          [ 5.7591, -2.4348, -2.9516]]], device='cuda:0'))